In [27]:
import requests
import json
import configparser
from time import time
# from pprint import pprint

In [7]:
config = configparser.ConfigParser()
config.read('../gnip_auth.ini')
api_user = config['GNIP_AUTH']['username']
api_passwd = config['GNIP_AUTH']['password']

In [8]:
url = "https://gnip-api.twitter.com/search/" \
        "fullarchive/accounts/greg-students/prod.json"

In [30]:
query = 'hello world'

# extended_query = query+" has:geo place_country:us"

params = {'query':query, 
          'maxResults': 500,
          'fromDate' : '201401010000',
          'toDate' : '201703200000'
         }

start = time()
response = requests.get(url, params=params, \
                 auth=(api_user, api_passwd))
end = time()
print(end-start)

1.1520607471466064


In [26]:
response.json()

{'next': 'eyJhdXRoZW50aWNpdHkiOiIxYWJlMDRlMzQyMzQxODhjZmJjNzgwNjMxZmJjMzJiZTEyMDM1MTc0OWY4MTJlM2RlNTE5YTc3N2I2M2JjNzM1IiwiZnJvbURhdGUiOiIyMDE0MDEwMTAwMDAiLCJ0b0RhdGUiOiIyMDE3MDMyMDAwMDAiLCJuZXh0IjoiMjAxNzAzMTkxOTMyMjgtODQzNTQ1NzI3MTYyODkyMjg3LTAifQ==',
 'requestParameters': {'fromDate': '201401010000',
  'maxResults': 10,
  'toDate': '201703200000'},
 'results': [{'actor': {'displayName': 'MADARA MCCREE✨',
    'favoritesCount': 83049,
    'followersCount': 1539,
    'friendsCount': 699,
    'id': 'id:twitter.com:30285088',
    'image': 'https://pbs.twimg.com/profile_images/853146062831276033/7UqZmhGF_normal.jpg',
    'languages': ['en'],
    'link': 'http://www.twitter.com/MicahVittoria',
    'links': [{'href': 'https://listography.com/adoniskun', 'rel': 'me'}],
    'listedCount': 137,
    'location': {'displayName': '(micah ☆ 19 ☆ she/her ☆ az)',
     'objectType': 'place'},
    'objectType': 'person',
    'postedTime': '2009-04-10T18:58:38.000Z',
    'preferredUsername': 'MicahVittor

**Using GNIP API search filters**

In [ ]:
class GnipData():
    
    def __init__(maxResults, geolocation, fromDate, toDate):
        self.maxResults = maxResults
        self.geolocation = geolocation
        self.fromDate = fromDate
        self.toDate = toDate
        self._cred = (api_user, api_pass)
        self.url = "https://gnip-api.twitter.com/search/" \
        "fullarchive/accounts/greg-students/prod.json"
    
    
    def fetchTweets(queries):
        params = {'fromDate':'201401010000',
                  'toDate': '201703200000',
                  'maxResults':self.maxResults}
        
        for query in queries:
            params['query'] = query
            
            response = requests.get(self.url,
                                    params=params,
                                    auth=self._cred)

            queueKafka(response.json()['results'])
            
            while ('next' in response.json().keys()):
                params['next'] = response.json()['next']
                
                response = requests.get(url,
                                        params=params,
                                        auth=self._cred)

                queueKafka(response.json()['results'])

    
    def queueKafka(json_data):
        # queue json data in kafka
        pass


In [ ]:
class SparkProcessing():
    
    def __init__(snopes_urls):
        self.urls = snopes_urls
    
    
    def runSpark():
        getQueries(self.urls)
    
    
    def getQueries(urls):
        # spark processing on urls to get keywords
        pass

In [ ]:
# process_urls = SparkProcessing(snopes_urls)
# queries = process_urls.runSpark()

# search = GnipData(100, 'Boulder', blah, blah)
# search.fetchTweets(queries)

In [ ]:
def fetchTweets(query, maxResults=500):
    # search API url
    url = "https://gnip-api.twitter.com/search/" \
    "fullarchive/accounts/greg-students/prod.json"
    
    # storing auth credentials in a hidden variable
    _cred=(api_user, api_pass)
    
    # setting parameters to append to the url
    params = {'query':query,
              'fromDate':'201401010000',
              'toDate': '201703200000',
              'maxResults': 500}
    
    # making a GET request to the API
    response = requests.get(url,
                            params=params,
                            auth=_cred)
    
    # storing the number of tweets returned
    reslen = len(response.json()['results'])
    
    # checking if more tweets are available
    while ('next' in response.json().keys()):

        yield reslen
        
        params['next'] = response.json()['next']

        response = requests.get(url,
                                params=params,
                                auth=_cred)

        reslen += len(response.json()['results'])

In [ ]:
# using search API word locality filter
for num_tweets in fetchTweets("obama thank you"):
    print(num_tweets)


In [7]:
import requests
import json
# import configparser
# from pykafka import KafkaClient

##parameters for the search
url ="https://gnip-api.twitter.com/search/fullarchive/accounts/greg-students/prod.json"
params = { 'query':'hillary', 'maxResults':'10',
'fromDate':'201701010000', 'toDate':'201701050000'}

_cred=(api_user, api_pass)

response = requests.get(url,
                        params=params,
                        auth=_cred)

r = response.json()['results'][0]

In [31]:
r = response.json()['results'][6]

In [32]:
r['place']

In [21]:
[tuple(ii['geo']['coordinates']) for ii in r['user']['derived']['locations']]

[(-81.30812, 28.69889)]

In [ ]:
import requests
import configparser
import json
from kafka import KafkaProducer

#We need better storage for all of these
#----------------------------------------
config = configparser.ConfigParser()
config.read('../gnip_auth.ini')
api_user = config['GNIP_AUTH']['username']
api_pass = config['GNIP_AUTH']['password']
TOPIC_NAME= "gnipstream"
HOST = "35.163.127.184"
KAFKA_PORT = "32777"
CASSANDRA_PORT = "9042"
#----------------------------------------


class GnipData():
    """
    Gets data from Gnip and pushes it to a Kafka queue. 
    Params-
    - maxResults (int): How many Results you want
    - maxResultsPerPage (int): Results per page, to get further we use the next param
    - fromDate (string- yyyymmddhhmmss) : Starting Date of all the tweets returned
    - toDate (string -yyyymmddhhmmss): Ending Date of all the tweets returned
    """
    def set_up(self):
        #self.TOPIC_NAME= "test4"
        #config = configparser.ConfigParser()
        #config.read('credentials.ini')
        self.url = "https://gnip-api.twitter.com/search/" \
            "fullarchive/accounts/greg-students/prod.json"
        #self.api_user = config['GNIP_API']['username']
        #self.api_passwd = config['GNIP_API']['password']
        self.kafka_server = KAFKA_ADDRESS
        self.kafka_producer = KafkaProducer(bootstrap_servers = [self.kafka_server])
        #this works in jupyter but not in terminal... working around
        # self.kafka_producer = KafkaProducer(value_serializer=lambda m: json.dumps(m).encode('unicode'),
        #                                    bootstrap_servers=[self.kafka_server])

    def __init__(self, maxResults, maxResultsPerPage):
        self.maxResults = maxResults
        self.maxResultsPerPage = maxResultsPerPage
        self.set_up()


    def __init__(self, maxResults, maxResultsPerPage, fromDate, toDate):
        self.maxResults = maxResults
        self.fromDate = fromDate
        self.toDate = toDate
        self.maxResultsPerPage = maxResultsPerPage
        self.set_up()

    def fetchTweets(self, query):
        """
        Takes a query and pushes the relevant tweets to Kafka
        Params
        - Query: Search term for Gnip
        
        Returns : Nothing
        """
        extended_query = query+" has:geo place_country:us"
        params = {'query':extended_query, 
                  'maxResults':self.maxResultsPerPage,
                  'fromDate' : self.fromDate,
                  'toDate' : self.toDate
                 }
        response = requests.get(self.url, params=params, \
                         auth=(api_user, api_passwd))

        for r in response.json()['results']:
                self.queueKafka( json.dumps(r).encode('utf-8'))

        #Scrolling through until next runs out or maxResults is exceeded
        for i in range(int(self.maxResults/self.maxResultsPerPage)):
            if 'next' in response.json().keys():
                params= {'query':extended_query, "next": response.json()['next']}
                response = requests.get(self.url, params=params, \
                             auth=(api_user, api_passwd))

                for r in response.json()['results']:
                        self.queueKafka( json.dumps(r).encode('utf-8'))
                #self.queueKafka(json.dumps(response.json()['results']).encode('utf-8'))
            else:
                break

    
    def queueKafka(self, json_data ):
        """
        Queues json to Kafka
        Params
        - json_data: Json data to queue in Kafka

        Returns : Nothing
        """
        self.kafka_producer.send(TOPIC_NAME, json_data)


if __name__ == "__main__":
    a = GnipData(11, 10 , "201612300000", "201612310000")
    a.fetchTweets("hillary")



In [ ]:
# from __future__ import print_function

import sys,json
from enum import Enum
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils
from pyspark.sql import Row, SparkSession

class Month(Enum):
    Jan='01'
    Feb='02'
    Mar='03'
    Apr='04'
    May='05'
    Jun='06'
    Jul='07'
    Aug='08'
    Sep='09'
    Oct='10'
    Nov='11'
    Dec='12'

def getSparkSessionInstance(sparkConf):
    if ('sparkSessionSingletonInstance' not in globals()):
        globals()['sparkSessionSingletonInstance'] = SparkSession.builder\
              .appName("SparkCassandraApp")\
              .config("spark.cassandra.connection.host", "172.32.13.183")\
              .config("spark.cassandra.connection.port", "9042")\
              .master("local[2]")\
              .getOrCreate();
    return globals()['sparkSessionSingletonInstance']


if __name__ == "__main__":
    if len(sys.argv) != 3:
        print("Usage: kafka_wordcount.py <zk> <topic>", file=sys.stderr)
        exit(-1)

    brokers, topic = sys.argv[1:]
    sc = SparkContext(appName="SparkCassandraApp")
    ssc = StreamingContext(sc, 1)

    kvs = KafkaUtils.createStream(ssc, brokers, "spark-streaming-consumer", {topic: 1})
    parsed = kvs.map(lambda x: json.loads(x[1]))

    def process(rdd):
        try:
            # Get the singleton instance of SparkSession
            spark = getSparkSessionInstance(rdd.context.getConf())

            # Convert RDD[String] to RDD[Row] to DataFrame
            rowRdd = rdd.map(lambda tweet: Row(id=tweet['id_str']))
            tweetsDataFrame = spark.createDataFrame(rowRdd)

            tweetsDataFrame.write\
                .format("org.apache.spark.sql.cassandra")\
                .options(table="test_tweets", keyspace="swashbucklers")\
                .save(mode="append")
            
            # Creates a temporary view using the DataFrame.
            #tweetsDataFrame.createOrReplaceTempView("tweets")

            # Do word count on table using SQL and print it
            #tweetidsDataFrame = \
            #    spark.sql("select id from test_tweets")
            #tweetidsDataFrame.show()

        except:
            pass

    parsed.foreachRDD(process)

    ssc.start()
    ssc.awaitTermination()
